In [43]:
# import internal .py modules
import file_path_management as fpath
import public_library as plib

In [44]:
# import packages
import csv
import pandas as pd
import PyPDF2
import requests
import time
import os
import random
from requests.auth import HTTPProxyAuth
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
import os
import re
from lxml import etree

In [46]:
# extract text from given .pdf file
def pdf2text(pdf_path, page_start, page_end):
    # creating a pdf reader object
    reader = PyPDF2.PdfReader(pdf_path)
    
    # printing number of pages in pdf file
    page_max = len(reader.pages)
    
    # getting a specific page from the pdf file
    text = ""
    
    for i in range(page_start, page_end + 1):
        # print(i)
        page = reader.pages[i]
        text = text + "".join(page.extract_text().splitlines())

    return text
# --------------------start of test code--------------------
# pdf_folder_path = fpath.litera_pdf_folder
# pdf_file_name = "1.pdf"
# pdf_path = os.path.join(pdf_folder_path, pdf_file_name)

# text_folder_path = fpath.litera_text_folder
# text_file_name = pdf_file_name.split(".pdf")[0] + ".txt"
# text_path = os.path.join(text_folder_path, text_file_name)

# start_page = 0
# end_page = 1
# text = pdf2text(pdf_path, start_page, end_page)

# with open(text_path, "w") as f:
#     f.write(text)

# f.close()
# ---------------------end of test code---------------------

In [47]:
# download pdf to specified folder given pdf_url and file name
def download_pdf(pdf_url, pdf_folder_path, file_name):    
    # get the pdf content
    options = webdriver.ChromeOptions()
    options.add_experimental_option('prefs', {
    "download.default_directory": pdf_folder_path, #Change default directory for downloads
    "download.prompt_for_download": False, #To auto download the file
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True #It will not show PDF directly in chrome
    })
    driver = webdriver.Chrome(options=options)
    response  = driver.get(pdf_url)
    time.sleep(5)
    # response = requests.get(pdf_url, stream=True, headers=plib.headers)
    
    # download the .pdf file to the pdf_file_path folder
    # write content in pdf file
    # pdf_path = os.path.join(pdf_folder_path, file_name + '.pdf')
    
    # if response.status_code == 200:
    #     with open(pdf_path, 'wb') as pdf_object:
    #         pdf_object.write(response.content)
    #         return True
    # else:
    #     print(f'Failed downloading PDF:' + 'pdf_url')
    #     print(f'HTTP response status code: {response.status_code}')
    #     return False
# --------------------start of test code--------------------
# pdf_url = 'https://www.sciencedirect.com/science/article/pii/S0896627320300052/pdfft?md5=3f0648c6385e6fae3a5a73b053903014&pid=1-s2.0-S0896627320300052-main.pdf'
# pdf_folder_path = fpath.litera_pdf_folder
# file_name = '4'
# download_pdf(pdf_url, pdf_folder_path, file_name)
# ---------------------end of test code---------------------

In [ ]:
# pubmed.ncbi.nlm.nih.gov
def func_pubmed_ncbi_nlm_nih_gov(url):
    # initialize
    info = {
        "doi": np.nan,
        "pmid": np.nan,
        "pmcid": np.nan,
        "title": np.nan,
        "abstract": np.nan,
        "keywords": np.nan,
        "pdf_link": np.nan
    }

    # load the webpage
    soup = plib.request_webpage(url)
    time.sleep(5)
    
    # extract information from loaded webpage
    # doi
    try:
        doi = soup.find_all("span", {"class": "identifier doi"})[0].find_all("a", {"class": "id-link"})[0].get_text().strip()
        doi = doi.strip()
    except:
        doi = np.nan
    if doi == doi:
        doi = doi.lower()
    # print(doi)

    # pmid
    try:
        pmid = soup.find("span", {"class": "identifier pubmed"}).find("strong", {"class": "current-id"}).get_text().strip()
        pmid = pmid.strip()
    except:
        pmid = np.nan
    # print(pmid)
    if pmid == pmid:
        pmid = str(int(pmid)).strip()
    # pmcid
    try:
        pmcid = soup.find("span", {"class": "identifier pmc"}).find("strong", {"class": "current-id"}).get_text().strip()
        pmcid = pmcid.strip()
    except:
        pmcid = np.nan
    # print(pmcid)

    # title
    try:
        title = soup.find("h1", {"class": "heading-title"}).get_text().strip()
        title = title.strip()
    except:
        title = np.nan
    # print(title)

    # abstract
    try:
        abstract = ""
        elems = soup.find(lambda tag: tag.name=="h2" and ("Abstract" in tag.text or "ABSTRACT" in tag.text or "Summary" in tag.text or "SUMMARY" in tag.text)).find_next_sibling("div").find_all("p")
        for elem in elems:
            abstract = abstract + " " + elem.get_text().strip()
        abstract = abstract.strip()
    except:
        try:
            abstract = ""
            elems = soup.find(lambda tag: tag.name=="h2" and ("Abstract" in tag.text or "ABSTRACT" in tag.text or "Summary" in tag.text or "SUMMARY" in tag.text)).find_next_siblings("p")
            for elem in elems:
                abstract = abstract + " " + elem.get_text().strip()
            abstract = abstract.strip()
        except:
            abstract = np.nan
    # print(abstract) 

    # keywords
    try:
        elems = soup.find(lambda tag:tag.name=="strong" and "Keywords" in tag.text)
        keywords = elems.findNext('span').get_text().strip()
        keywords = keywords.strip()
    except:
        keywords = np.nan
    # print(keywords)

    # pdf_link
    try:
        pdf_link = soup.find_all("li", {"class": "pdf-link other_item"})[0].find_all("a")[0]["href"]
        pdf_link = "https://www.ncbi.nlm.nih.gov" + pdf_link
        pdf_link = pdf_link.strip()
    except:
        pdf_link = np.nan
    # print(pdf_link)

    info = {
        "doi": doi,
        "pmid": pmid,
        "pmcid": pmcid,
        "title": title,
        "abstract": abstract,
        "keywords": keywords,
        "pdf_link": pdf_link
    }

    return info
# --------------------start of test code--------------------
# # url = "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10133512/"
# # url = "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2613515/"
# # url = "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8328208/"
# # url = "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8541979/"
# # url = "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4855639/"
# # url = "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3140205/"
# # url = "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4362213/"
# url = "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC48909/"
# info = func_ncbi_nlm_nih_gov(url)
# print(info["doi"])
# print(info["pmid"])
# print(info["pmcid"])
# print(info["title"])
# print(info["abstract"])
# print(info["keywords"])
# print(info["pdf_link"])
# ---------------------end of test code---------------------